<a href="https://colab.research.google.com/github/matthew-mcateer/GradientBidding/blob/master/PriceOfAnarchy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Price of Anarchy

In [0]:
import tensorflow as tf
import types
import random
import numpy
from numpy import linalg as LA
numpy.set_printoptions(precision=3, suppress=True)


In [0]:
def alloc(mode, contrib, hparams):
    graph = tf.Graph()
    session = tf.Session(graph=graph)
    
    # Build optimization problem.
    with graph.as_default():
        
        # Contribution: W* :  The True (i,j)-utility  
        Contribution = tf.constant(contrib, tf.float32)

        # Weights: W: Contribution row of W.
        weights_list = []
        for _ in range(n):
            weights_i = tf.Variable(tf.random.uniform((1, hparams.n_nodes), minval=0))
            weights_list.append(weights_i)
        Weights = tf.concat(weights_list, axis=0)
        
        # We force weights onto [0,1] using the sigmoid and then normalize.
        Weights_sig = tf.sigmoid(Weights)
        Weights_norm = tf.linalg.normalize(Weights_sig, ord=1, axis=1)[0]
        
        # Weights_diag: Is the matrix with the main diagonal of Weights. a.k.a self-contribution.
        Weights_diag = tf.matrix_set_diag(tf.zeros((hparams.n_nodes, hparams.n_nodes)), tf.linalg.tensor_diag_part(Weights_norm), k = 0)

        # Interranking: Q: The inter-model ranking derived from the weights. We use the infinite series
        # absorbing markov chain calculation. 
        Interranking = tf.linalg.inv(tf.eye(hparams.n_nodes) - Weights_norm + Weights_diag)
        
        Interranking_mean = tf.reshape(tf.tile(tf.reduce_mean(Interranking, axis=0), [hparams.n_nodes]), [hparams.n_nodes, hparams.n_nodes])
        Divergence = tf.nn.sigmoid_cross_entropy_with_logits(labels=Interranking_mean, logits=Interranking)
        
        # Emission: E :The quantity of additional stake attributed to the nodes.
        Emission = tf.linalg.tensor_diag_part(Interranking * Weights_diag)

        # Market_Contribution: The allocation function shifted contribution scores given the bids.
        Market_Shift = tf.reduce_mean(Weights_norm, axis=1)
        Masked_Contribution = tf.multiply(Contribution, tf.nn.relu(Weights_norm - Market_Shift))
        
        # Utility: U: The utility gained from the loss function given the masked contributions.
        Utility = tf.reduce_sum(Masked_Contribution, axis=1)

        # Payoff: P: Utility + Emission
        Payoff = Utility + Emission - hparams.divergence_gamma *tf.reduce_sum(Divergence, axis=1)
        
        ### Bellow Optimization.

        # Bidders move in the direction of the gradient of the Payoff.
        optimizer = tf.train.AdamOptimizer(hparams.learning_rate)
        
        # Mode == Competitive: All nodes optimize only their local payoff. 
        train_steps = []
        if mode == 'competitive':
            for i in range(hparams.n_nodes):
                payoff_i = tf.slice(Payoff, [i], [1])
                weights_i = weights_list[i]
                grads_and_vars_i = optimizer.compute_gradients(loss=-payoff_i, var_list=[weights_i])
                train_steps.append(optimizer.apply_gradients(grads_and_vars_i))

        # Mode == Coordinated: Coordinated nodes optimize the aggregated payoff
        elif mode == 'coordinated':
            grads_and_vars = optimizer.compute_gradients(loss=-tf.reduce_mean(Payoff), var_list=weights_list)
            train_steps.append(optimizer.apply_gradients(grads_and_vars))

        # Init the graph.
        session.run(tf.global_variables_initializer())

        # Converge...
        for step in range(hparams.n_steps):
            
            # Randomly choose participant to optimize
            if mode == 'competitive':
                step = random.choice(train_steps)
                
            # Optimize all participants.
            elif mode == 'coordinated':
                step = train_steps[0]
                
            # Run graph.
            output = session.run(fetches = 
                                      {   
                                        'step': step,       
                                        'Payoff': Payoff,   
                                        'Utility': Utility, 
                                        'Emission': Emission,
                                        'Divergence': Divergence,
                                        'Weights': Weights_norm,  
                                        'Mask': Masked_Contribution,
                                        'Interranking': Interranking,
                                      })      
        # Return metrics.
        return output


In [0]:
hparams = types.SimpleNamespace( 
    trials = 1,
    n_steps = 1,
    learning_rate = 0.05,
    n_nodes = 10,
    contrib_factor = 1,
    divergence_gamma=0.01
)


for _ in range(hparams.trials):
    
    # Build "True" contribution matrix.
    contribution = numpy.random.randn(hparams.n_nodes, hparams.n_nodes)
    contribution = (contribution - numpy.min(contribution))/numpy.ptp(contribution)
    contribution = contribution/contribution.sum(axis=1, keepdims=1) * hparams.contrib_factor
    
    print ('Contribution: W*')
    print (contribution)
    print ('')
    
    # Run coordinated weight convergence.
    coord_output = alloc('coordinated', contribution, hparams)
    
    # Run competitive weight convergence.
    comp_output = alloc('competitive', contribution, hparams)
    
    print ('Coordinated Mask: M')
    print (coord_output['Mask'])
    print ('')
    
    print ('Competitive Mask: M')
    print (comp_output['Mask'])
    print ('')

    print ('Coordinated Weights: W')
    print (coord_output['Weights'])
    print ('')
    
    print ('Competitive Weights: W')
    print (comp_output['Weights'])
    print ('')
    
    print ('Coordinated Interranking: Q')
    print (coord_output['Interranking'])
    print ('')
    
    print ('Competitve Interranking: Q')
    print (comp_output['Interranking'])
    print ('')
    
    print ('Coordinated Divergence: D')
    print (coord_output['Divergence'])
    print ('')
    
    print ('Competitive Divergence: D')
    print (comp_output['Divergence'])
    print ('')
    
    print ('Coordinated Emission: E')
    print (coord_output['Emission'])
    print ('')
    
    print ('Competitive Emission: E')
    print (comp_output['Emission'])
    print ('')
    
    print ('Coordinated Utility: U')
    print (coord_output['Utility'])
    print ('')
    
    print ('Competitive Utility: U')
    print (comp_output['Utility'])
    print ('')
    
    print ('Coordinated Payoff: P')
    print (coord_output['Payoff'])
    print ('Avg:' + str(sum(coord_output['Payoff'])/hparams.n_nodes))
    print ('')
    
    print ('Competitive Payoff: P')
    print (comp_output['Payoff'])
    print ('Avg:' + str(sum(comp_output['Payoff'])/hparams.n_nodes))
    print ('')

    poa = sum(comp_output['Payoff']) / sum(coord_output['Payoff'])
    print ('Price of Anarchy: ' + str(poa))



Contribution: W*
[[0.107 0.106 0.045 0.103 0.116 0.083 0.086 0.067 0.12  0.168]
 [0.079 0.084 0.145 0.037 0.12  0.097 0.079 0.171 0.062 0.127]
 [0.078 0.117 0.132 0.114 0.086 0.083 0.15  0.137 0.08  0.023]
 [0.011 0.131 0.065 0.129 0.081 0.107 0.122 0.052 0.19  0.114]
 [0.103 0.036 0.1   0.146 0.092 0.138 0.077 0.102 0.074 0.131]
 [0.175 0.149 0.078 0.087 0.106 0.108 0.145 0.038 0.046 0.067]
 [0.072 0.154 0.082 0.101 0.1   0.078 0.107 0.101 0.099 0.106]
 [0.132 0.06  0.11  0.161 0.051 0.052 0.031 0.131 0.157 0.115]
 [0.088 0.068 0.135 0.087 0.119 0.11  0.152 0.059 0.058 0.125]
 [0.097 0.126 0.114 0.065 0.112 0.114 0.134 0.    0.112 0.127]]

Coordinated Mask: M
[[0.    0.    0.001 0.    0.    0.    0.    0.001 0.001 0.001]
 [0.    0.001 0.002 0.    0.    0.    0.    0.    0.001 0.   ]
 [0.001 0.    0.    0.001 0.    0.001 0.    0.002 0.    0.   ]
 [0.    0.    0.001 0.    0.    0.    0.    0.001 0.002 0.   ]
 [0.001 0.    0.001 0.    0.001 0.    0.    0.    0.001 0.   ]
 [0.001 0.    0.